# Any Imput Required by the Program is submitted by making changes to the Yaml File, editing it is as easy as editing a notepad

# Importing the Necessary Python Libraries

In [111]:
import yaml
import pandas as pd
from urllib import request
from datetime import datetime, timedelta
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile
from urllib.error import HTTPError, URLError
from socket import timeout
import logging
from pathlib import Path
import os

# Reading the Yaml file for all the required Inputs

In [112]:
with open(r'C:\Users\91888\Desktop\input.yaml') as file:
    inp=yaml.load(file, Loader=yaml.FullLoader)

# Creating Directories for the Raw Data & the Final Data (Data with 1 month Performance per SYMBOL

In [113]:
Path(inp['Output_Src_Data']).mkdir(parents=True, exist_ok=True)
Path(inp['Output_Final_Data']).mkdir(parents=True, exist_ok=True)

# Creating Class and Class Methods for Reading & Data Transformations

In [114]:
class nse_extract:
    
    def __init__(self,date,url,no_of_days=30):
        self.date=pd.to_datetime(date)
        self.days=no_of_days
        self.url=url 
        self.ll=[] 
        
    def list_of_files(self,date):
        for i in range(0,self.days+2,1):
            dt=self.date
            dt=dt-timedelta(days=i)
            y=str(dt.year)
            m=dt.strftime('%b').upper()
            d=str(dt.day)
            if len(d)==1:    #URL required day in date to be double digit, Eg, '08', '09', '15'
                d="0"+d
            if dt.isoweekday() in range(1, 6):  # The Data is only available on Weekdays
                self.ll.append(self.url+y+'/'+m+'/cm'+d+m+y+'bhav.csv.zip') # Creating the URLs for each data, since the pattern changes everyday
            else:
                continue
 
    #Reading the Zipped Data driectly from the URL and loading the raw data in Source Directory
    
    def get_days_data_for_symbol(self,Output=inp['Output_Src_Data']):
        self.output_loc=Output
        for i in self.ll:
            zipurl = i
            try:        # Handling Errors due to Session Time Out or URL Error
                with urlopen(zipurl, timeout=10) as zipresp:
                    with ZipFile(BytesIO(zipresp.read())) as zfile:
                        zfile.extractall(Output)   
            except (HTTPError, URLError) as error:
                logging.error('Data not retrieved because URL: %s',zipurl)
            except timeout:
                logging.error('socket timed out - URL: %s', zipurl)
    
    #Creating the Final data sources, where we have one file for each SYMBOL's performance in last 30 days
    
    def final_data_for_symbol(self, Input=inp['Output_Src_Data'], Output=inp['Output_Final_Data']):
        file_names=os.listdir(Input)
        df0=pd.DataFrame()
        df1=pd.DataFrame()
        for i in file_names:
            df0=pd.read_csv(Input+'\\'+i,sep=',')
            df1=df1.append(df0)
        df1=df1.iloc[:,:-1]
        for i in df1.SYMBOL.unique():
            df1[df1.SYMBOL==i].to_csv(Output+'\\'+i+'.csv',index=False)

# Creating an instance of the class

In [115]:
obj=nse_extract(inp['Date'],inp['url'],inp['No_of_days_data'])

# Creating the list of last one month's data

In [116]:
obj.list_of_files(inp['Date'])

# Creating the Source Data

In [117]:
obj.get_days_data_for_symbol(inp['Output_Src_Data'])

ERROR:root:socket timed out - URL: https://archives.nseindia.com/content/historical/EQUITIES/2020/FEB/cm21FEB2020bhav.csv.zip


# Creating the Final Data 

In [118]:
obj.final_data_for_symbol(inp['Output_Src_Data'],inp['Output_Final_Data'])